In [26]:
import pandas as pd
import numpy as np
import pickle

def save(model, filename='bestmodel.pickle'):
    with open('output/'+filename, 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
def submit(model):
    model = model.fit(train_x,train_y)
    pred = model.predict(final_test)
    final_test['SalePrice'] = np.exp(pred)
    final_test[['Id','SalePrice']].to_csv('output/submission.csv', index=False)

In [27]:
#make train and test datasets. Splitting labels and features happens later
path_train = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/train.csv"
path_test = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/test.csv"

train = pd.read_csv(path_train)
final_test = pd.read_csv(path_test)
print(train.shape, final_test.shape)

y_col = (set(train.columns) - set(final_test.columns)).pop()
train[y_col] = np.log1p(train[y_col])

# since we use cross validation the train set does not have to be split anymore
# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(
#     train.drop([y_col], axis=1),train.loc[:,y_col], test_size=0.33, random_state=42) 
# ds = (train_x,test_x,train_y,test_y)
# for d in ds: print(d.shape)
train_x = train.drop([y_col], axis=1)
train_y = train.loc[:,y_col]
train_x.shape, train_y.shape


(1460, 81) (1459, 80)


((1460, 80), (1460,))

### Preprocessing

In [28]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta


In [29]:
from sklearn.impute import SimpleImputer
# todo multivariate imputation, possibly with pipelines for numeric and categorical data

from sklearn.preprocessing import MinMaxScaler
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
# https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler/51237727
# don't know features are normal so just going with minmax scalar atm

from sklearn.preprocessing import OneHotEncoder
#https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
#The crux of it is that the sklearn encoder creates a function which persists and can then be applied to new data sets which use the same categorical variables, with consistent results.
# So don't use pandas get dummies, but a OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# We create the preprocessing pipelines for both numeric and categorical data.
corr = train.corr()[y_col]
corr = corr.sort_values(ascending=False)
num_x = list(corr.index)[1:]

numeric_transformer = Pipeline(steps=[
    ('impute_num', SimpleImputer(strategy='median')),
    ('scale_num', MinMaxScaler())])

cat_x = [col for col in final_test.columns if final_test[col].dtype == 'object']
cat_x.sort(key = lambda x: -correlation_ratio(train[x],train[y_col]))

categorical_transformer = Pipeline(steps=[
    ('impute_cat', SimpleImputer(strategy='most_frequent')),
    ('onehot_cat', OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, num_x),
        ('category', categorical_transformer, cat_x)])

### Modeling

In [32]:

# from catboost import CatBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet,SGDRegressor,GammaRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_validate
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def inv_y(transformed_y):
    return np.exp(transformed_y)


models = [('RidgeCV',RidgeCV()),
            ('svm.SVR',svm.SVR()),
            ('LinearRegression',LinearRegression()),
            ('LassoCV',LassoCV()),
            ('ElasticNet',ElasticNet()),
            ('GammaRegressor',GammaRegressor()),
            ('KNeighborsRegressor',KNeighborsRegressor()),
            ('RandomForestRegressor',RandomForestRegressor()),
            ('SGDRegressor',SGDRegressor()),
            ('CatBoostRegressor',CatBoostRegressor(silent=True)),
            ('xgb.XGBRegressor',xgb.XGBRegressor()),
            ('lgb.LGBMRegressor',lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)),
            ('HistGradientBoostingRegressor',HistGradientBoostingRegressor()),
            ('MLPRegressor',MLPRegressor())
]

def get_pipeline(model):
    return Pipeline([('preprocess', preprocessor),
                   ('model',model)])

for name, model in models:
    pipe = get_pipeline(model)
    # score = -1 * cross_val_score(pipe, train_x, train_y,cv=3,scoring='neg_root_mean_squared_error')
    # below is necessary for looking at train scores
    num_fold = 3
    scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
    # scoring is identical to 
    # make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
    # neg_mean_squared_log_error
    print(f"{name}, \ttrain {-1 * sum(scores['train_score'])/num_fold:.3f}, test {-1 * sum(scores['test_score'])/num_fold:.3f}")

ridge, 	train 0.115, test 0.143
svm, 	train 0.088, test 0.141
lin, 	train 0.089, test 0.151
lasso, 	train 0.109, test 0.137
ElasticNet, 	train 0.399, test 0.399
GammaRegressor, 	train 0.248, test 0.251
KNeighborsRegressor, 	train 0.169, test 0.215
RandomForestRegressor, 	train 0.056, test 0.147
SGDRegressor, 	train 0.196, test 0.233
CatBoostRegressor, 	train 0.032, test 0.122
xgb.XGBRegressor, 	train 0.005, test 0.150
lgb.LGBMRegressor, 	train 0.190, test 0.211
HistGradientBoostingRegressor, 	train nan, test nan
MLPRegressor, 	train 0.138, test 0.196


In [46]:
from sklearn.model_selection import GridSearchCV

hyperparameters = { 'preprocess__category__impute_cat__strategy': ['most_frequent','constant'],
                    'preprocess__numeric__impute_num__strategy': ['mean','median'],
                  }

hyperparameters = {'alpha': [1,2,3],
                    'lambda': [1,2,3]}
clf = GridSearchCV(get_pipeline(xgb.XGBRegressor()), hyperparameters, cv=2)
 
# Fit and tune model
clf.fit(train_x, train_y)

KeyError: 'base_score'

## Saving

In [ ]:
train_info = (train_x, train_y, num_x, cat_x)
with open('output/train_info.pickle', 'wb') as handle:
    pickle.dump(train_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
model = get_pipeline(xgb.XGBRegressor())
submit(model)
save(model)

In [0]:
m